In [1]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize
import scipy.signal


In [2]:

CALIBRATION_FILE = r"C:\Users\emwhit\Dropbox\projects\mag-fingers\pymagnets\calibrations\helmholz_18_11_12_15_09_31.txt"

PER_CHANNEL_GAIN = True
CROSSTALK_MATRIX = True
CROSS_BIAS = False


def apply_calibration(data, per_channel_gain, crosstalk, cross_bias):
    calibrated = np.matmul(data, np.diag(per_channel_gain))
    calibrated = np.sqrt(np.matmul(calibrated**2, crosstalk))
    calibrated = np.sqrt(calibrated**2 + cross_bias)
    return calibrated


def apply_calibration_x(data, x):
    per_channel_gain, crosstalk, cross_bias = decode_x(x)
    return apply_calibration(data, per_channel_gain, crosstalk, cross_bias)


def measure_error(calibrated):
    mag = np.linalg.norm(calibrated, axis=1)
    error = mag - 1

    # print(matrix, bias, np.mean(error**2))
    return np.mean(error**2)*100


def filter_bad_data(data):
    print(data.shape)
    filtered = scipy.signal.savgol_filter(data, 15, 2, axis=0)
    error = np.sqrt(np.sum((data-filtered)**2, axis=1))
    # plt.figure()
    # plt.plot(data)
    # plt.figure()
    # plt.plot(error)
    # plt.show()
    return data[error < .007, :]


def encode_x(per_channel_gain, crosstalk, cross_bias):
    x = []
    if PER_CHANNEL_GAIN:
        x += per_channel_gain
    if CROSSTALK_MATRIX:
        x += list(crosstalk.flatten())
    if CROSS_BIAS:
        x += cross_bias
    return np.array(x)


def decode_x(x):
    i = 0

    if PER_CHANNEL_GAIN:
        per_channel_gain = x[i:i+3]
        i += 3
    else:
        per_channel_gain = [1, 1, 1]

    if CROSSTALK_MATRIX:
        crosstalk = x[i:i+9].reshape(3, 3)
        i += 9
    else:
        crosstalk = np.eye(3)

    if CROSS_BIAS:
        cross_bias = x[i:i+3]
        i += 3
    else:
        cross_bias = [0, 0, 0]

    return per_channel_gain, crosstalk, cross_bias


def get_bounds():
    bounds = []
    if PER_CHANNEL_GAIN:
        bounds += [(1,3)] * 3

    if CROSSTALK_MATRIX:
        for i in range(9):
            if i % 4 == 0:
                bounds += [(.999, 1.001)]
            else:
                bounds += [(-.5, 0)]

    if CROSS_BIAS:
        bounds += [(-.2, 0)] * 3

    return bounds


In [3]:
print("Loading data...")
data = np.loadtxt(CALIBRATION_FILE, delimiter=",")

data = filter_bad_data(data)
# data = data[np.abs(data[:,0]) > .1]
plt.plot(np.abs(data))

Loading data...
(119600, 3)


c:\users\emwhit\appdata\local\programs\python\python37\lib\site-packages\scipy\signal\_arraytools.py:45: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  b = a[a_slice]


FigureCanvasNbAgg()

In [27]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display


fig = plt.figure(figsize=(15, 9), dpi=80)
ax = fig.add_subplot(111)
hs = ax.plot(data)
# ax.set_ylim((0, .2))
# ax.set_xlim((10000, 30000))
def f(x, y):
    per_channel_gain = [1.73543888, 1.73543888, 1.73543888]
    crosstalk = np.eye(3)
    crosstalk[0,1] = x
    crosstalk[2,1] = y
    cross_bias = [1, 1, 1]
    x0 = encode_x(per_channel_gain, crosstalk, cross_bias)
    calibrated2 = apply_calibration_x(data, x0)
    for i, handle in enumerate(hs):
        handle.set_ydata(calibrated2[:,i])
    
interact(f, x=(-0.01, 0.0,.0001), y=(-0.01, 0.0,.0001))

plt.show()



FigureCanvasNbAgg()

interactive(children=(FloatSlider(value=-0.005, description='x', max=0.0, min=-0.01, step=0.0001), FloatSlider…